In [30]:
import pandas as pd
import numpy as np
from pandas import Timedelta
from datetime import timedelta
from sklearn.datasets import load_wine
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
df_20_col = pd.read_parquet('../../models/KMeans/output/seatunnal_20col.parquet')
df_original = pd.read_parquet('../../Sonar/seatunnel_all_information.parquet')

In [32]:
df = pd.concat([df_20_col, df_original['total_time']], axis=1)

In [33]:
date = pd.to_timedelta(df['total_time'])
time_hours = date.dt.total_seconds() / 3600

df['hours'] = time_hours

In [34]:
result_time  = []

for i in df['hours']:
    if i > 153.70 :
        result_time.append(0)
    elif (i>=48.54) & (i<=153.70):
        result_time.append(1)
    else:
        i <= 48.54
        result_time.append(2)
df['time_class'] = result_time

In [35]:
df.head(5)

,begin_D,begin_B,begin_CP,begin_C,begin_OOA,end_D,end_B,end_CP,end_C,end_OOA,...,C_change,OOA_change,D_percent,B_percent,CP_percent,C_percent,OOA_percent,total_time,hours,time_class
0,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,...,0.0,-389.0,0.000000,0.381679,0.381679,0.0,-127.960526,3 days 01:27:48,73.463333,1
1,500.0,262.0,262.0,304.0,304.0,501.0,262.0,262.0,304.0,693.0,...,0.0,-389.0,-0.200000,0.000000,0.000000,0.0,-127.960526,3 days 04:15:10,76.252778,1
2,500.0,262.0,262.0,304.0,304.0,499.0,261.0,261.0,304.0,693.0,...,0.0,-389.0,0.200000,0.381679,0.381679,0.0,-127.960526,0 days 05:24:03,5.400833,2
3,500.0,262.0,262.0,304.0,304.0,500.0,261.0,261.0,304.0,693.0,...,0.0,-389.0,0.000000,0.381679,0.381679,0.0,-127.960526,4 days 00:22:26,96.373889,1
4,499.0,261.0,261.0,304.0,304.0,500.0,262.0,262.0,304.0,693.0,...,0.0,-389.0,-0.200401,-0.383142,-0.383142,0.0,-127.960526,12 days 00:39:27,288.657500,0


In [36]:
X = df.drop(['total_time', 'hours', 'time_class'], axis=1)
y = df['time_class']

In [37]:
# X = df.loc[:, ~df.columns.isin([
#                                 'hours', 'scored' , 'clusters' , 
#                                 r'[\'begin_Bloaters\', \'begin_Couplers\', \'begin_Object-Orientation Abusers\']_3' ,
#                                 'total_time' ,
#                                 'begin_sha', 'end_sha', 'begin_time' , 'begin_key', 'begin_revision', 'end_time', 'end_key', 'end_revision'
#                                 ])]
# y = df['time_class']

In [38]:
# x_drop = X.columns[-2]

In [39]:
# X.drop(x_drop, axis=1)

In [40]:
X_train, X_test = X[:int(0.7 * len(X))], X[int(0.7 * len(X)):]
y_train, y_test = y[:int(0.7 * len(X))], y[int(0.7 * len(X)):]

In [41]:
int(0.7 * len(X))

747

In [42]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1 , max_depth=10 , random_state=3).fit(X_train , y_train)

In [43]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.15420561, 0.06542056, 0.06542056, 0.25821596, 0.657277  ])

In [44]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([0, 0, 0, ..., 2, 2, 2])

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y , y_pred)

array([[ 40,   9, 109],
       [ 74,   7, 113],
       [287, 220, 209]])

In [46]:
clf.predict(X)

array([1, 1, 2, ..., 2, 2, 2])

In [47]:
clf.predict(X_train)

array([1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2,
       0, 2, 2, 0, 1, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 0, 1, 2, 0, 2, 0, 2, 0, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0,
       2, 2, 1, 2, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2,
       1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 1, 1, 0, 0, 2, 2, 1, 1,
       0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2,
       2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0,
       0, 1, 2, 2, 2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 2,

In [48]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred ,average='macro'))
print('recall_score = ', recall_score(y,y_pred ,average='macro'))
print('f1_score',f1_score(y,y_pred ,average='macro'))

precision =  0.20477681129800976
recall_score =  0.19371549084320386
f1_score 0.18003326195374206


In [49]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         1626
y_pred    1098
dtype: int64

In [50]:
from sklearn.model_selection import GridSearchCV

parameters = {

    'learning_rate': [0.01 , 0.1 ,1.0],
    'n_estimators': [1,12,14,16],
    'subsample' : [0.7],
    'max_depth' : [1,2] ,
    'random_state':[2]

}
GradientBoosting = GradientBoostingClassifier()
clf = GridSearchCV(GradientBoosting, parameters , cv=3 , scoring='f1_macro')
clf.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [1, 2],
                         'n_estimators': [1, 12, 14, 16], 'random_state': [2],
                         'subsample': [0.7]},
             scoring='f1_macro')

In [51]:
y_pred = cross_val_predict(clf.best_estimator_,X_test,y_test)
y_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [52]:
y_pred_clf = clf.best_estimator_.predict(X_test)
y_pred_clf

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y_test,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y_test,y_pred_clf ,average='macro'))
print('f1_score',f1_score(y_test,y_pred_clf ,average='macro'))

precision =  0.26375908618899274
recall_score =  0.3333333333333333
f1_score 0.29449275362318844


/Users/bill/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
import itertools

In [56]:
# result = itertools.combinations(X ,2)
# 
# for item in result:
#     df  = pd.DataFrame(item)
#     X[list (item)]
#     parameters = {
#     'learning_rate': [0.01 , 0.1 ,1],
#     'n_estimators': [12,14,16],
#     'subsample' : [0.7],
#     'max_depth' : [2] ,
#     'random_state':[2]
#     }
#     GradientBoosting = GradientBoostingClassifier()
#     clf = GridSearchCV(GradientBoosting , parameters , cv=3 , scoring='f1_macro')
#     clf.fit(X_train[list (item)],y_train)
#     clf.predict(X_test[list (item)])
#     print('predict=',clf.predict(X_test[list (item)]))
#     print(X_test)
#     print("y_train = ",y_train)
#     print(item)
#     print("precision_score =",precision_score(clf.predict(X[list(item)]),y))
#     print("recall_score =", recall_score(clf.predict(X[list(item)]),y))
#     print("f_1=",f1_score(clf.predict(X[list(item)]),y,average='macro'))
#     print("f_1_X-test=",f1_score(clf.predict(X_test[list(item)]),y_test,average='macro'))

In [57]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [58]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
reg_nnls = LinearRegression(positive=True)
y_pred_nnls = reg_nnls.fit(X_train, y_train).predict(X_test)
r2_score_nnls = r2_score(y_test, y_pred_nnls)
print("NNLS R2 score", r2_score_nnls)

NNLS R2 score -0.033463583087028015
